In [26]:
from mediawikiapi import MediaWikiAPI
mediawikiapi = MediaWikiAPI()
page = mediawikiapi.page("ad astra (film)")
content = page.content
print(content)

# get the content between '== Cast ==' and next topic
content = content.split('== Cast ==')[1].split('==')[0]
print(content)

Ad Astra is a 2019 American psychological science fiction film produced, co-written, and directed by James Gray. Starring Brad Pitt (who also produced), Tommy Lee Jones, Ruth Negga, Liv Tyler, and Donald Sutherland, it follows an astronaut who ventures into space in search of his lost father, whose obsessive quest to discover intelligent alien life at all costs threatens the Solar System and all life on Earth. The project was announced in early 2016, with Gray saying he wanted to feature "the most realistic depiction of space travel that's been put in a movie". Pitt signed on to star in April 2017 and the rest of the cast joined later that year. Filming began around Los Angeles that August, lasting through October.
Ad Astra premiered at the Venice Film Festival on August 29, 2019, and was theatrically released in the United States on September 20, 2019, by Walt Disney Studios Motion Pictures via 20th Century Fox. It received positive reviews from critics, with praise for Pitt's perform

In [4]:
import requests
import json
import pandas as pd
import sqlite3

# url encoding library
import urllib.parse

api_key = 'c84d2eacda018a9f013d10a6880631ac'

# CONNECT TO movie_corpus.db
conn = sqlite3.connect('movie_corpus.db')
cursor = conn.cursor()

# GET ALL CHARACTERS GROUPED BY MOVIE
cursor.execute('SELECT name, title FROM character JOIN movie WHERE movie.id = character.movie_id GROUP BY title')

# convert to url encoding
title = urllib.parse.quote_plus(title)

print(title)

url = f'https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={title}'

resp = requests.get(url).json()

print(resp)

id = resp['results'][0]['id']

url = f'https://api.themoviedb.org/3/movie/{id}?api_key={api_key}&append_to_response=credits'

resp = requests.get(url).json()

#Print readable JSON
print(json.dumps(resp, indent=4, sort_keys=True))

NameError: name 'title' is not defined

In [2]:
import requests
import json
import pandas as pd
import sqlite3

# url encoding library
import urllib.parse

api_key = 'c84d2eacda018a9f013d10a6880631ac'

# CONNECT TO movie_corpus.db
conn = sqlite3.connect('final.db')
cursor = conn.cursor()

# GET ALL CHARACTERS GROUPED BY MOVIE
characterFilm = cursor.execute('SELECT name, title FROM character JOIN movie WHERE movie.id = character.movie_id ORDER BY title').fetchall()

titles = cursor.execute('SELECT title FROM movie ORDER BY title').fetchall()

# CREATE MOVIE - CHARACTERS DICTIONARY
movieCharactersDatabase = {}
movieCharactersAPI = {}
movieActorsAPI = {}



In [3]:
for title in titles:
    
    title = title[0]
    print(f'Title - {title}')
    
    movieCharactersDatabase[title] = []
    movieCharactersAPI[title] = []
    movieActorsAPI[title] = []
    
    # convert to url encoding
    titleParsed = urllib.parse.quote_plus(title)
    
    print(f'TitleParsed - {titleParsed}')
    
    url = f'https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={titleParsed}'
    resp = requests.get(url).json()
    id = resp['results'][0]['id']
    url = f'https://api.themoviedb.org/3/movie/{id}?api_key={api_key}&append_to_response=credits'
    resp = requests.get(url).json()
    
    # GET CHARACTERS FROM CAST
    cast = resp['credits']['cast']
    for character in cast:
        movieCharactersAPI[title].append(character['character'])
        movieActorsAPI[title].append((character['name'], character['character']))

Title - 10 things i hate about you
TitleParsed - 10+things+i+hate+about+you
Title - 1492: conquest of paradise
TitleParsed - 1492%3A+conquest+of+paradise
Title - 15 minutes
TitleParsed - 15+minutes
Title - 2001: a space odyssey
TitleParsed - 2001%3A+a+space+odyssey
Title - 48 hrs.
TitleParsed - 48+hrs.
Title - 8mm
TitleParsed - 8mm
Title - a bucket of blood
TitleParsed - a+bucket+of+blood
Title - a clockwork orange
TitleParsed - a+clockwork+orange
Title - a hard day's night
TitleParsed - a+hard+day%27s+night
Title - a nightmare on elm street
TitleParsed - a+nightmare+on+elm+street
Title - a nightmare on elm street 3: dream warriors
TitleParsed - a+nightmare+on+elm+street+3%3A+dream+warriors
Title - a nightmare on elm street 4: the dream master
TitleParsed - a+nightmare+on+elm+street+4%3A+the+dream+master
Title - a nightmare on elm street part 2: freddy's revenge
TitleParsed - a+nightmare+on+elm+street+part+2%3A+freddy%27s+revenge
Title - a nightmare on elm street: the dream child
Title

In [7]:
# CREATE ACTORS TABLE
cursor.execute('DROP TABLE IF EXISTS actor')
cursor.execute('CREATE TABLE actor (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT UNIQUE);')
conn.commit()

# POPULATE ACTORS TABLE

for title in movieActorsAPI:
    for actor,character in movieActorsAPI[title]:
        try:
            cursor.execute('INSERT INTO actor (name) VALUES (?)', (actor,))
        except:
            pass
        
conn.commit()

# ADD ACTOR_ID TO CHARACTERS TABLE
cursor.execute('ALTER TABLE character ADD COLUMN actor_id INTEGER REFERENCES actor (id) ON DELETE CASCADE;')
conn.commit()

OperationalError: duplicate column name: actor_id

In [8]:
# ITERATE THROUGH ALL MOVIES
for character in characterFilm:
     
    # GET MOVIE TITLE
    title = character[1]
    
    # GET CHARACTERS
    movieCharactersDatabase[title].append(character[0])
    
# ITERATE THROUGH ALL movieCharactersDatabase

In [1]:
from thefuzz import fuzz
from thefuzz import process
countEmptyString = 0

for title in movieCharactersDatabase:
    # GET CHARACTERS
    charactersDatabase = movieCharactersDatabase[title]
    charactersAPI = movieCharactersAPI[title]
    
    for character in charactersDatabase:
        
        if character == None:
            print(f'{title} - {character}, {characterAPI}')
            
        characterCopy = character
        character = character.lower().replace('.', '').replace('-', '')
        
        maxFuzzRatio = 0
        bestCharAPI = ''
        
        for characterAPI in charactersAPI:
            
            characterAPICopy = characterAPI
            characterAPI = characterAPI.lower().replace('.', '').replace('-', '')
                        
            if characterAPI == '':
                continue
            
            fuzzRatio = fuzz.ratio(character, characterAPI)
            
            if fuzzRatio > 90 or character in characterAPI or characterAPI in character:
                
                if fuzzRatio > maxFuzzRatio:
                    maxFuzzRatio = fuzzRatio
                    bestCharAPI = characterAPICopy
                    
                    for actor, char in movieActorsAPI[title]:
                        if char == bestCharAPI:
                            actor_id = cursor.execute('SELECT id FROM actor WHERE name = ?', (actor,)).fetchone()[0]
                            movie_id = cursor.execute('SELECT id FROM movie WHERE title = ?', (title,)).fetchone()[0]
                            character_id = cursor.execute('SELECT id FROM character WHERE name = ? AND movie_id = ?', (characterCopy, movie_id)).fetchone()[0]
                            cursor.execute('UPDATE character SET actor_id = ? WHERE id = ?', (actor_id, character_id))
                            conn.commit()
                    
                
    
        
        if bestCharAPI == '':
            countEmptyString += 1
            continue
                
print(f'Empty Strings - {countEmptyString}')


NameError: name 'movieCharactersDatabase' is not defined